# 물류센터 입지선정 - 최종 ver

In [1]:
import os
os.chdir(r"C:\Users\myhyu\OneDrive\Desktop\2024-2 BaF")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
#pip install pulp
import pulp

#pip install geopy
from geopy.distance import geodesic
import folium

---

# 데이터셋
모두 2차 전처리까지 완료된 데이터셋 
- 상파울루 : SP
    - 47134 rows
    

- 바이아 : BA
    - 3396 rows

### 상파울루 - SP
상파울루의 면적은 서울의 약 2.5배  
상파울루의 인구는 약 1,300만 명

In [2]:
df_sp = pd.read_csv('전처리2차_SP.csv')
df_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41107 entries, 0 to 41106
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       41107 non-null  object 
 1   order_status                   41107 non-null  object 
 2   customer_id                    41107 non-null  object 
 3   customer_unique_id             41107 non-null  object 
 4   customer_zip_code_prefix       41107 non-null  int64  
 5   customer_city                  41107 non-null  object 
 6   customer_state                 41107 non-null  object 
 7   geolocation_lat                41107 non-null  float64
 8   geolocation_lng                41107 non-null  float64
 9   order_purchase_timestamp       41107 non-null  object 
 10  order_approved_at              41101 non-null  object 
 11  order_delivered_carrier_date   40808 non-null  object 
 12  order_delivered_customer_date  40475 non-null 

In [3]:
# 중심 좌표 설정 (평균 위도, 경도)
latitude = df_sp['geolocation_lat'].mean()
longitude = df_sp['geolocation_lng'].mean()

# Folium 지도 생성
m_sp = folium.Map(location=[latitude, longitude], zoom_start=10)

# 고객 위치를 지도에 마커로 표시 (투명도 설정)
for index, row in df_sp.iterrows():
    folium.CircleMarker([row['geolocation_lat'], row['geolocation_lng']],
                        radius=3,
                        color='blue',
                        fill=True,
                        fill_opacity=0.5  # 투명도 설정 (0.0에서 1.0 사이 값)
                       ).add_to(m_sp)

# 지도를 HTML 파일로 저장
m_sp.save('sp_customer_map_outlierX.html')

In [4]:
coords_sp = df_sp[['geolocation_lat', 'geolocation_lng']].values

# K-means Clustering
1차 클러스터링 :5

In [11]:
# K=5
kmeans = KMeans(n_clusters=5, random_state=0)
df_sp['cluster'] = kmeans.fit_predict(coords_sp)
df_sp['cluster'].value_counts()

cluster
0    26407
3     7024
2     3398
1     2231
4     2047
Name: count, dtype: int64

In [12]:
# 클러스터 중심 좌표 출력
cluster_centers = kmeans.cluster_centers_
print("클러스터 중심 좌표:")
for i, center in enumerate(cluster_centers):
    print(f"클러스터 {i}: 위도: {center[0]}, 경도: {center[1]}")

클러스터 중심 좌표:
클러스터 0: 위도: -23.588587878712463, 경도: -46.624547099900404
클러스터 1: 위도: -21.71665804914388, 경도: -50.49214998731062
클러스터 2: 위도: -21.350336461341964, 경도: -48.36640231928193
클러스터 3: 위도: -22.919464357699315, 경도: -47.334611568987754
클러스터 4: 위도: -23.148800258124083, 경도: -45.60890783624817


In [6]:
dbi_combined = davies_bouldin_score(df_sp[['geolocation_lat', 'geolocation_lng']], df_sp['cluster'])
print(f"통합된 클러스터링 DBI: {dbi_combined}")

통합된 클러스터링 DBI: 0.7197276982942069


In [7]:
# 클러스터 별 고객 좌표 및 수
clusters = df_sp['cluster'].unique()  # 모든 클러스터의 고유 값
coords_all_clusters = [df_sp[df_sp['cluster'] == cluster][['geolocation_lat', 'geolocation_lng']].values for cluster in clusters]

# MILP 최적화

In [8]:
# 각 클러스터의 고객 수
num_customers_per_cluster = [len(coords) for coords in coords_all_clusters]
num_centers = 5  # 물류센터 수

# 고객과 물류센터 간의 거리 계산 함수 (유클리드 거리)
def calculate_distance(customer_loc, center_loc):
    return np.linalg.norm(customer_loc - center_loc)

# 고객-센터 간 거리 행렬 생성
cost_matrix = []
for cluster_index, coords in enumerate(coords_all_clusters):
    cluster_costs = np.zeros((num_customers_per_cluster[cluster_index], num_centers))
    for customer_index in range(num_customers_per_cluster[cluster_index]):
        for center_index in range(num_centers):
            # 클러스터 센터 좌표를 이용한 거리 계산
            cluster_costs[customer_index][center_index] = calculate_distance(coords[customer_index], kmeans.cluster_centers_[center_index])
    cost_matrix.append(cluster_costs)

# MILP 문제 정의
problem = pulp.LpProblem("Fulfillment_Center_Location_Clustering", pulp.LpMinimize)


In [9]:
# 고객-센터 간 서비스 여부를 나타내는 이진 변수 x[i][k][j]
x = [
    [[pulp.LpVariable(f"x_{i}_{k}_{j}", cat='Binary') for j in range(num_centers)]
      for k in range(num_customers_per_cluster[i])]
    for i in range(len(clusters))
]

# 물류센터 설치 여부를 나타내는 이진 변수 y[j]
y = [pulp.LpVariable(f"y_{j}", cat='Binary') for j in range(num_centers)]

# 목적 함수: 총 배송 비용 최소화
problem += pulp.lpSum(cost_matrix[i][k][j] * x[i][k][j]
                      for i in range(len(clusters))
                      for j in range(num_centers)
                      for k in range(num_customers_per_cluster[i]))

# 제약 조건 1: 각 고객은 하나의 물류센터에서만 서비스 받아야 한다
for i in range(len(clusters)):
    for k in range(num_customers_per_cluster[i]):
        problem += pulp.lpSum(x[i][k][j] for j in range(num_centers)) == 1

# 제약 조건 2: 물류센터가 설치된 경우에만 고객이 해당 센터에서 서비스를 받을 수 있다
for j in range(num_centers):
    for i in range(len(clusters)):
        for k in range(num_customers_per_cluster[i]):
            problem += x[i][k][j] <= y[j]

# 제약 조건 3: 물류센터 용량 제한 (예시: 각 물류센터는 최대 20,000명의 고객 처리 가능)
capacity = 20000
for j in range(num_centers):
    problem += pulp.lpSum(x[i][k][j] for i in range(len(clusters)) for k in range(num_customers_per_cluster[i])) <= capacity * y[j]


In [10]:
# 최적화 문제 해결
problem.solve()

# 설치된 물류센터 출력
for j in range(num_centers):
    if pulp.value(y[j]) == 1:
        print(f"물류센터 {j} 설치됨.")
    else:
        print(f"물류센터 {j} 설치되지 않음.")

# 고객 할당 결과 출력
for i in range(len(clusters)):
    for k in range(num_customers_per_cluster[i]):
        for j in range(num_centers):
            if pulp.value(x[i][k][j]) == 1:
                print(f"클러스터 {i}의 고객 {k}는 물류센터 {j}에서 서비스를 받음.")

물류센터 0 설치됨.
물류센터 1 설치됨.
물류센터 2 설치됨.
물류센터 3 설치됨.
물류센터 4 설치됨.
클러스터 0의 고객 0는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 1는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 2는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 3는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 4는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 5는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 6는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 7는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 8는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 9는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 10는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 11는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 12는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 13는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 14는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 15는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 16는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 17는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 18는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 19는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 20는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 21는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 22는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 23는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 24는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 25는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 26는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 27는 물류센터 0에서 서비스를 받음.
클러스터 0의 고객 28는 물류센터 0에서 서비

# 입지선정 결과

In [ ]:
# 지도의 중심 좌표 설정
latitude = df_sp['geolocation_lat'].mean()
longitude = df_sp['geolocation_lng'].mean()

# Folium 지도 생성
m = folium.Map(location=[latitude, longitude], zoom_start=12)

# 클러스터 색상 정의
colors = ['blue', 'green', 'orange', 'purple', 'red']  # cluster 0-4 색상 정의

# 고객 위치를 지도에 마커로 표시
for cluster_id in range(5):
    cluster_data = df_sp[df_sp['cluster'] == cluster_id]
    for index, row in cluster_data.iterrows():
        color = colors[cluster_id]
        folium.CircleMarker([row['geolocation_lat'], row['geolocation_lng']],
                            radius=3,
                            color=color,
                            fill=True).add_to(m)

# 클러스터 센터 표시
centers = []
for cluster_id in range(5):
    center = kmeans.cluster_centers_[cluster_id]
    folium.Marker([center[0], center[1]], popup=f'Center {cluster_id}',
                  icon=folium.Icon(color='black')).add_to(m)
    centers.append(center)

# 지도 출력 및 저장
m.save("sp_fulfillment_centers_final.html")


In [52]:
df_sp.to_csv('입지선정완료1_sp.csv', index=False)